In [2]:
### ValDXer testing
import os
os.environ["HDXER_PATH"] = "/home/alexi/Documents/HDXer"
from ValDX.ValidationDX import ValDXer
from ValDX.VDX_Settings import Settings
import pandas as pd
import MDAnalysis as mda
from MDAnalysis.coordinates.XTC import XTCWriter


settings = Settings()
settings.replicates = 3
settings.gamma_range = (2,6)
settings.train_frac = 0.5
settings.RW_exponent = [0]
settings.split_mode = 'R3'
settings.stride = 1000
# settings.HDXer_stride = 10000

settings.RW_do_reweighting = False
settings.RW_do_params = True
import pickle

# VDX = ValDXer(settings)


In [3]:
# import subprocess
# from ValDX.helpful_funcs import conda_to_env_dict

# # Assuming settings.HDXer_env contains the name of your Conda environment
# env_path = conda_to_env_dict(settings.HDXer_env)

# command = "echo $HDXER_PATH"
# print("command:", command)

# # Run the command in the subprocess
# output = subprocess.run(command, shell=True, env=env_path, capture_output=True, text=True)

# # Capture and print the standard output (stdout)
# hdxer_path = output.stdout.strip()  # .strip() removes any trailing newline
# print("HDXER_PATH:", hdxer_path)


In [4]:
def pre_process_main_BPTI():
    # BPTI data
    expt_name = 'Experimental'
    test_name = "MD_Simulated"

    BPTI_dir = "/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/HDXer_tutorial/BPTI"
    BPTI_dir = "/home/alexi/Documents/ValDX/raw_data/HDXer_tutorial/BPTI"
    # BPTI_dir = "/home/alexi/Documents/ValDX/raw_data/HDXer_tutorial/BPTI"
    expt_dir = os.path.join(BPTI_dir, "BPTI_expt_data")

    os.listdir(expt_dir)

    segs_name = "BPTI_residue_segs.txt"
    segs_path = os.path.join(expt_dir, segs_name)

    hdx_name = "BPTI_expt_dfracs.dat"
    hdx_path = os.path.join(expt_dir, hdx_name)
    print(hdx_path)

    rates_name = "BPTI_Intrinsic_rates.dat"
    rates_path = os.path.join(expt_dir, rates_name)
    sim_name = 'BPTI_MD'

    sim_dir = os.path.join(BPTI_dir, "BPTI_simulations")

    os.listdir(sim_dir)

    md_reps = 1
    rep_dirs = ["Run_"+str(i+1) for i in range(md_reps)]

    top_name = "bpti_5pti_eq6_protonly.gro"

    top_path = os.path.join(sim_dir, rep_dirs[0], top_name)

    traj_name = "bpti_5pti_reimg_protonly.xtc"

    traj_paths = [os.path.join(sim_dir, rep_dir, traj_name) for rep_dir in rep_dirs]

    print(top_path)
    print(traj_paths)


    small_traj_name = traj_name.replace(".xtc","_small.xtc")
    small_traj_path = os.path.join(sim_dir, small_traj_name)

    u = mda.Universe(top_path, traj_paths)

    
        
    with XTCWriter(small_traj_path, n_atoms=u.atoms.n_atoms) as W:
        for ts in u.trajectory[:5]:
                W.write(u.atoms)

    traj_paths = [small_traj_path]
    

    return hdx_path, segs_path, rates_path, top_path, traj_paths, sim_name, expt_name, test_name


In [5]:
hdx_path, segs_path, rates_path, top_path, traj_paths, sim_name, expt_name, test_name = pre_process_main_BPTI()

/home/alexi/Documents/ValDX/raw_data/HDXer_tutorial/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat
/home/alexi/Documents/ValDX/raw_data/HDXer_tutorial/BPTI/BPTI_simulations/Run_1/bpti_5pti_eq6_protonly.gro
['/home/alexi/Documents/ValDX/raw_data/HDXer_tutorial/BPTI/BPTI_simulations/Run_1/bpti_5pti_reimg_protonly.xtc']


In [6]:
# def run_split_test(split_mode, name, system):

#     # settings.split_mode = 'R'
#     settings.split_mode = split_mode
#     settings.name = "_".join([name, split_mode, system])

#     VDX = ValDXer(settings)

#     VDX.load_HDX_data(HDX_path=hdx_path, SEG_path=segs_path, calc_name=expt_name)
#     VDX.load_intrinsic_rates(path=rates_path, calc_name=expt_name)

#     VDX.load_structures(top_path=top_path, traj_paths=traj_paths, calc_name=test_name)

#     run_outputs = VDX.run_VDX(calc_name=test_name, expt_name=expt_name)
#     analysis_dump, df, name = VDX.dump_analysis()
#     save_path = VDX.save_experiment()

#     return run_outputs, analysis_dump, df, name, save_path

In [7]:
def run_split_test_BPTI(split_mode, name, system):

    # settings.split_mode = 'R'
    settings.split_mode = split_mode
    settings.name = "_".join([name, split_mode, system])

    VDX = ValDXer(settings)

    VDX.load_HDX_data(HDX_path=hdx_path, SEG_path=segs_path, calc_name=expt_name)
    VDX.load_intrinsic_rates(path=rates_path, calc_name=expt_name)

    VDX.load_structures(top_path=top_path, traj_paths=traj_paths, calc_name=test_name)

    run_outputs = VDX.run_VDX(calc_name=test_name, expt_name=expt_name)
    analysis_dump, df, name = VDX.dump_analysis()
    save_path = VDX.save_experiment()

    return run_outputs, analysis_dump, df, name, save_path


splits = ['r', 's', 'R3', 'xR', 'Sp', 'SR']
split_names = ['naiverandom', 'NC_Termini', 'RedundantK', 'XtureK', 'pINspace', 'spaceK']
system = 'BPTITtut'

raw_run_outputs = {}
analysis_dumps = {}
analysis_df = pd.DataFrame()
names = []
save_paths = []


for split, split_name in zip(splits, split_names):
    run_outputs, analysis_dump, df, name, save_path = run_split_test_BPTI(split, split_name, system)
    raw_run_outputs[name] = run_outputs
    analysis_dumps.update(analysis_dump)
    analysis_df = pd.concat([analysis_df, df])
    names.append(name)
    save_paths.append(save_path)



envs ['# conda environments:', '#', 'base                     /home/alexi/anaconda3', 'HDXER_ENV             *  /home/alexi/anaconda3/envs/HDXER_ENV', 'openfold_env             /home/alexi/anaconda3/envs/openfold_env', 'plumed-masterclass-2022     /home/alexi/anaconda3/envs/plumed-masterclass-2022', '                         /home/alexi/localcolabfold/colabfold-conda', '                         /home/alexi/localcolabfold/conda', '']
name base
name HDXER_ENV
Environment 'HDXER_ENV' found.
Path to 'HDXER_ENV' environment: /home/alexi/anaconda3/envs/HDXER_ENV
PATH /home/alexi/anaconda3/envs/HDXER_ENV:/home/alexi/anaconda3/envs/HDXER_ENV/bin:/home/alexi/anaconda3/condabin:/home/alexi/.vscode/cli/servers/Stable-903b1e9d8990623e3d7da1df3d33db3e42d80eda/server/bin/remote-cli:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin
calc_hdx
/home/alexi/Documents/HDXer/HDXer/calc_hdx.py
Experiment name naiverandom_r_BPTITtut already exists. At

/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:241: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn("Reload offsets from trajectory\n "


envs ['# conda environments:', '#', 'base                     /home/alexi/anaconda3', 'HDXER_ENV             *  /home/alexi/anaconda3/envs/HDXER_ENV', 'openfold_env             /home/alexi/anaconda3/envs/openfold_env', 'plumed-masterclass-2022     /home/alexi/anaconda3/envs/plumed-masterclass-2022', '                         /home/alexi/localcolabfold/colabfold-conda', '                         /home/alexi/localcolabfold/conda', '']
name base
name HDXER_ENV
Environment 'HDXER_ENV' found.
Path to 'HDXER_ENV' environment: /home/alexi/anaconda3/envs/HDXER_ENV
PATH /home/alexi/anaconda3/envs/HDXER_ENV:/home/alexi/anaconda3/envs/HDXER_ENV/bin:/home/alexi/anaconda3/condabin:/home/alexi/.vscode/cli/servers/Stable-903b1e9d8990623e3d7da1df3d33db3e42d80eda/server/bin/remote-cli:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin
calc_hdx
/home/alexi/Documents/HDXer/HDXer/calc_hdx.py
conda run -n HDXER_ENV python /home/alexi/Documents/HDXer

/home/alexi/Documents/HDXer/HDXer/dfpred.py:505: RuntimeWarning: divide by zero encountered in log
  logerr = logf + np.log(err) + np.log(time) # sd(e^Aa) = f * sd(A) * a
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/alexi/anaco

/home/alexi/Documents/ValDX/data/naiverandom_r_BPTITtut70/train_MD_Simulated_1/reweighting_gamma_3x10^0/home/alexi/Documents/ValDX/data/naiverandom_r_BPTITtut70/train_MD_Simulated_1/reweighting_gamma_4x10^0/home/alexi/Documents/ValDX/data/naiverandom_r_BPTITtut70/train_MD_Simulated_1/reweighting_gamma_2x10^0/home/alexi/Documents/ValDX/data/naiverandom_r_BPTITtut70/train_MD_Simulated_1/reweighting_gamma_5x10^0



Contacts readContacts readContacts read


Hbonds readHbonds readHbonds read


Segments and experimental dfracs readSegments and experimental dfracs readSegments and experimental dfracs read


Contacts read
Hbonds read
Segments and experimental dfracs read
Finished reweighting
[(array([0.2, 0.2, 0.2, 0.2, 0.2]), 0.24016185180444222, 4.535566134647084), (array([0.2, 0.2, 0.2, 0.2, 0.2]), 0.24016185180444222, 4.535566134647084), (array([0.2, 0.2, 0.2, 0.2, 0.2]), 0.24016185180444222, 4.535566134647084), (array([0.2, 0.2, 0.2, 0.2, 0.2]), 0.24016185180444222, 4.535566134647084)]
RW

...

/home/alexi/Documents/ValDX/data/spaceK_SR_BPTITtut18/train_MD_Simulated_3/reweighting_gamma_3x10^0/home/alexi/Documents/ValDX/data/spaceK_SR_BPTITtut18/train_MD_Simulated_3/reweighting_gamma_2x10^0/home/alexi/Documents/ValDX/data/spaceK_SR_BPTITtut18/train_MD_Simulated_3/reweighting_gamma_4x10^0/home/alexi/Documents/ValDX/data/spaceK_SR_BPTITtut18/train_MD_Simulated_3/reweighting_gamma_5x10^0



Contacts readContacts readContacts readContacts read



Hbonds readHbonds readHbonds read
Hbonds read


Segments and experimental dfracs readSegments and experimental dfracs readSegments and experimental dfracs read


Segments and experimental dfracs read
Finished reweighting
[(array([0.2, 0.2, 0.2, 0.2, 0.2]), 0.04607979677501451, 10.308220185252573), (array([0.2, 0.2, 0.2, 0.2, 0.2]), 0.04607979677501451, 10.308220185252573), (array([0.2, 0.2, 0.2, 0.2, 0.2]), 0.04607979677501451, 10.308220185252573), (array([0.2, 0.2, 0.2, 0.2, 0.2]), 0.04607979677501451, 10.308220185252573)]
RW_do_reweight

/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/MDAnalysis/core/universe.py:817: DeprecationWarning: The bfactor topology attribute is only provided as an alias to the tempfactor attribute. It will be removed in 3.0. Please use the tempfactor attribute instead.
  warnings.warn(BFACTOR_WARNING, DeprecationWarning)
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/MDAnalysis/core/topologyattrs.py:282: DeprecationWarning: The bfactor topology attribute is only provided as an alias to the tempfactor attribute. It will be removed in 3.0. Please use the tempfactor attribute instead.
  warnings.warn(BFACTOR_WARNING, DeprecationWarning)
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:

['train_MD_Simulated_1', 'train_MD_Simulated_2', 'train_MD_Simulated_3']
['val_MD_Simulated_1', 'val_MD_Simulated_2', 'val_MD_Simulated_3']
plotting dfracs compare for train
plot_dfracs_compare
       0.167      1.0     10.0  120.0  peptide            calc_name  ResStr  \
0    0.15669  0.63957  0.99996    1.0        0         Experimental     NaN   
1    0.09039  0.43293  0.99656    1.0        1         Experimental     NaN   
2    0.02665  0.14933  0.80157    1.0        2         Experimental     NaN   
3    0.18680  0.71010  1.00000    1.0        3         Experimental     NaN   
4    0.99880  1.00000  1.00000    1.0        4         Experimental     NaN   
..       ...      ...      ...    ...      ...                  ...     ...   
245  1.00000  1.00000  1.00000    1.0       36  test_MD_Simulated_3    51.0   
246  1.00000  1.00000  1.00000    1.0       37  test_MD_Simulated_3    52.0   
247  1.00000  1.00000  1.00000    1.0       38  test_MD_Simulated_3    53.0   
248  1.00000  1.

/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/alexi/anaconda3/envs/HDXER_ENV/lib/python3.8/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


plotting R agreement
plotting paired trainval agreement
       0.167      1.0     10.0  120.0  peptide           calc_name  ResStr  \
0    0.15669  0.63957  0.99996    1.0        0        Experimental     NaN   
1    0.09039  0.43293  0.99656    1.0        1        Experimental     NaN   
2    0.02665  0.14933  0.80157    1.0        2        Experimental     NaN   
3    0.18680  0.71010  1.00000    1.0        3        Experimental     NaN   
4    0.99880  1.00000  1.00000    1.0        4        Experimental     NaN   
..       ...      ...      ...    ...      ...                 ...     ...   
122  1.00000  1.00000  1.00000    1.0       36  val_MD_Simulated_3    51.0   
123  1.00000  1.00000  1.00000    1.0       37  val_MD_Simulated_3    52.0   
124  1.00000  1.00000  1.00000    1.0       38  val_MD_Simulated_3    53.0   
125  1.00000  1.00000  1.00000    1.0       39  val_MD_Simulated_3    54.0   
126  1.00000  1.00000  1.00000    1.0       40  val_MD_Simulated_3    55.0   

     Re

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named df
# Replace 'your_dataframe' with your actual DataFrame variable
df = analysis_df

# Create a FacetGrid, using 'name' for each subplot
g = sns.FacetGrid(df, col="name", col_wrap=3, height=4, aspect=1.5)
g.fig.suptitle('MSE over Time by Type for each Named Split Mode')

# Create boxplots
g = g.map(sns.boxplot, "time", "mse", "Type", palette="Set3")

# Adding some additional options for better visualization
g.add_legend(title='Type')
g.set_axis_labels("Time", "MSE")
g.set_titles("{col_name}")

# Adjust the arrangement of the plots
plt.subplots_adjust(top=0.9)

# Show plot
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named df
# Replace 'your_dataframe' with your actual DataFrame variable
df = analysis_df

# Create a FacetGrid, using 'name' for each subplot
g = sns.FacetGrid(df, col="name", col_wrap=3, height=4, aspect=1.5)
g.fig.suptitle('R over Time by Type for each Named Split Mode')

# Create boxplots
g = g.map(sns.boxplot, "time", "R", "Type", palette="Set3")

# Adding some additional options for better visualization
g.add_legend(title='Type')
g.set_axis_labels("Time", "R")
g.set_titles("{col_name}")

# Adjust the arrangement of the plots
plt.subplots_adjust(top=0.9)

# Show plot
plt.show()


In [ ]:
# plot LogPfs by Residues colour by calc_name facet wrap by name
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
LogPfs = pd.concat([analysis_dumps[i]["LogPfs"] for i in names])

print(LogPfs)

LogPfs_df = LogPfs.explode(['LogPf','Residues'])


# Create a FacetGrid, using 'name' for each subplot
g = sns.FacetGrid(LogPfs_df, col="name", col_wrap=3, height=4, aspect=1.5)
g.fig.suptitle('LogPfs over Residues for each Named Split Mode')

# Create lineplots
g = g.map(sns.lineplot, "Residues", "LogPf", "calc_name", palette="Set2")

# Adding some additional options for better visualization
g.add_legend(title='calc_name')
g.set_axis_labels("Residues", "LogPf")
g.set_titles("{col_name}")

# Adjust the arrangement of the plots
plt.subplots_adjust(top=0.9)

# Show plot
plt.show()


In [ ]:
# plot BV distributions 
# print(analysis_dumps["AvsB_S"]["BV_constants"])

BV_constants = pd.concat([analysis_dumps[i]["BV_constants"] for i in names])
num_names = len(BV_constants)/len(names)
BV_constants["name"] = [i for i in names for j in range(int(num_names))]
print(BV_constants)

# Create a FacetGrid, using 'name' for each subplot - plot as scatter plot
g = sns.FacetGrid(BV_constants, col="name", col_wrap=3, height=4, aspect=1.5)
g.fig.suptitle('BV Constants over Residues for each Named Split Mode')

# Create lineplots
g = g.map(sns.scatterplot, "Bc", "Bh", "calc_name", palette="Set2")

# Adding some additional options for better visualization
g.add_legend(title='calc_name')
g.set_axis_labels("Residues", "BV")
g.set_titles("{col_name}")

# Adjust the arrangement of the plots
plt.subplots_adjust(top=0.9)

# Show plot
plt.show()


In [ ]:
# plot BV distributions 
bc, bh = 0.35, 2

BV_constants = pd.concat([analysis_dumps[i]["BV_constants"] for i in names])
num_names = len(BV_constants)/len(names)
BV_constants["name"] = [i for i in names for j in range(int(num_names))]
print(BV_constants)

# Create a FacetGrid with a single axis
g = sns.FacetGrid(BV_constants, height=4, aspect=1.5)
g.fig.suptitle('BV Constants over Residues for each Named Split Mode')

# Create scatter plot
g = g.map(sns.scatterplot, "Bc", "Bh", "name", palette="Set2", alpha=0.5)

# Adding dashed lines at bc and bh
plt.axhline(bh, color='grey', linestyle='dashed')
plt.axvline(bc, color='grey', linestyle='dashed')

# Adding some additional options for better visualization
g.add_legend(title='name')
g.set_axis_labels("Bc", "Bh")

# Adjust the arrangement of the plots
plt.subplots_adjust(top=0.9)

# Show plot
plt.show()


In [ ]:
# print(top_path)

In [ ]:
# pdb_test = mda.Universe(top_path)

# # write out as a pdb and add header
# pdb_test.atoms.write('test.pdb')
# with open('test.pdb', 'r') as original: data = original.read()
# with open('test.pdb', 'w') as modified: modified.write('HEADER    '+sim_name+'\n'+data)

